# Configure file

Since this file will also be uploaded, this avoids exposing sensitive data by configuration global paths

In [6]:
import configparser

In [7]:
configuration = configparser.ConfigParser()
configuration.read('project.ini')

['project.ini']

In [8]:
IMAGES_FOLDER_PATH = configuration.get('DEFAULT', 'images_folder')

## Django setup

In [4]:
import django
import os
import pathlib
from django.conf import settings

In [5]:
BASE_DIR = os.path.abspath('.')

In [6]:
settings.configure(
    BASE_DIR=BASE_DIR,
    INSTALLED_APPS=[
        'django.contrib.admin',
        'django.contrib.auth',
        'django.contrib.contenttypes',
        'django.contrib.sessions',
        'django.contrib.messages',
        'django.contrib.staticfiles',

        'django_extensions',
        'corsheaders',
        'rest_framework',
        'rest_framework.authtoken',

        'api',
        'shop',
        'cart',
        'orders',
        'shipments',
        'reviews',
    ],
    DATABASES={
        'default': {
            'ENGINE': 'django.db.backends.sqlite3',
            'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
        }
    },
    MEDIA_ROOT=os.path.join(BASE_DIR, 'media'),
    DEFAULT_AUTO_FIELD='django.db.models.BigAutoField',
    PRODUCT_MODEL='shop.Product',
    CUSTOMER_ORDERS_MODEL='orders.CustomerOrder',
    STRIPE_TOKENS=[()]
)


In [7]:
os.environ.setdefault('DJANGO_ALLOW_ASYNC_UNSAFE', 'True')

'True'

In [8]:
django.setup()

## File collection

In [9]:
def collect_files():
    if os.path.exists(IMAGES_FOLDER_PATH):
        result = list(os.walk(IMAGES_FOLDER_PATH))
        full_path, dirs, files = result[0]
        
        images = filter(lambda x: any([x.endswith('jpg'), x.endswith('jpeg')]), files)
        videos = filter(lambda x: x.endswith('mp4'), files)
        
        return full_path, list(images), list(videos)
    return None, [], []

## Product file generation
Functions to generate a generic set of products from collected named fashion images

In [16]:
import json
import pathlib
import re
from shop.utils import create_product_slug
import random


In [17]:
def detect_media_name(name):
    """Detects the true name of the media file"""
    result = re.match(r'(\w+\s)+', name)
    if result is not None:
        return result.group(0).strip().lower()


In [18]:
def create_product_fixtures(model_dotted_path):
    """Creates a bunch of generic products for the test database"""
    with open(os.path.join('.', 'fixtures.json'), mode='w', encoding='utf-8') as f:
        products = []

        _, images, _ = collect_files()
        names = set(map(detect_media_name, images))

        for i, name in enumerate(names):
            if name is not None:
                product = {
                    'name': name.title(),
                    'color': random.choice(['Red' , 'Blue']),
                    'unit_price': random.randrange(10, 800),
                    'active': False,
                    'display_new': random.choice([True, False]),
                    'slug': create_product_slug(name),
                    'category': random.choice(['Skirts', 'Activewear', 'Panties'])
                }

                is_on_sale = random.choice([True, False])
                if is_on_sale:
                    product['sale_value'] = random.randrange(10, 50)
                    product['on_sale'] = True
                else:
                    product['sale_value'] = 0
                    product['on_sale'] = False
                    
                products.append({
                    'model': model_dotted_path,
                    'pk': i + 1,
                    'fields': product
                })

        json.dump(products, f)
        print('Created', len(names), 'products')
        return products


create_product_fixtures('shop.Product')

Created 40 products


[{'model': 'shop.Product',
  'pk': 1,
  'fields': {'name': 'Blazer Strapped',
   'color': 'Blue',
   'unit_price': 562,
   'active': False,
   'display_new': False,
   'slug': 'blazer-strapped',
   'category': 'Skirts',
   'sale_value': 27,
   'on_sale': True}},
 {'model': 'shop.Product',
  'pk': 2,
  'fields': {'name': 'Floral Mesh Balcony Soutien Gorge Et String Ensemble',
   'color': 'Red',
   'unit_price': 744,
   'active': False,
   'display_new': True,
   'slug': 'floral-mesh-balcony-soutien-gorge-string-ensemble',
   'category': 'Panties',
   'sale_value': 0,
   'on_sale': False}},
 {'model': 'shop.Product',
  'pk': 3,
  'fields': {'name': 'Jupe Florale',
   'color': 'Red',
   'unit_price': 307,
   'active': False,
   'display_new': False,
   'slug': 'jupe-florale',
   'category': 'Skirts',
   'sale_value': 12,
   'on_sale': True}},
 {'model': 'shop.Product',
  'pk': 4,
  'fields': {'name': 'Short En Jean',
   'color': 'Red',
   'unit_price': 573,
   'active': False,
   'display

In [ ]:
from shop.choices import ColorChoices, CategoryChoices

In [45]:
# There are 36 groups of images so we'll
# be creating a total of 36 products

def generate_generic_products():
    with open(os.path.join(IMAGES_FOLDER_PATH, 'fashion.json'), mode='w', encoding='utf-8') as f:
        products = []
        
        _, images, _ = collect_files()
        names = set(map(detect_media_name, images))
       
        for name in names:
            if name is not None:
                product = {
                    'name': name.title(),
                    'color': random.choice(ColorChoices.choices)[-1],
                    'unit_price': random.randrange(10, 800),
                    'active': False,
                    'display_new': random.choice([True, False]),
                    'slug': create_product_slug(name),
                    'category': random.choice(CategoryChoices.choices)[-1]
                }
                
                is_on_sale = random.choice([True, False])
                if is_on_sale:
                    product['sale_value'] = random.randrange(10, 50)
                    product['on_sale'] = True
                else:
                    product['sale_value'] = 0
                    product['on_sale'] = False
                products.append(product)
                
        json.dump(products, f)
        print('Created', len(names), 'products')
        return products


In [46]:
PRODUCTS = generate_generic_products()

Created 40 products


## Database objects creation
Create all the test items from above in the database

In [48]:
from shop.models import Image, Video, Product
from shop.utils import process_file_name
from django.contrib.auth import get_user_model
from django.core.files import File


In [60]:
def create_database_objects(commit=False, images_alone=False):
    full_path, images, videos = collect_files()
        
    def image_iterator():
        for i, image in enumerate(images):
            f = open(os.path.join(full_path, image), mode='rb')
            base_name, _, _ = process_file_name(image)
            clean_name = detect_media_name(base_name)
            yield Image(name=f"{clean_name}{i}", original=File(f, name=image))
            
    def video_iterator():
        for i, video in enumerate(videos):
            f = open(os.path.join(full_path, video), mode='rb')
            base_name, _, _ = process_file_name(video)
            clean_name = detect_media_name(base_name)
            yield Video(name=f"{clean_name}{i}", content=File(f, name=video))

    if commit:
        Image.objects.bulk_create(image_iterator())
        
        if not images_alone:
            Video.objects.bulk_create(video_iterator())

In [61]:
def create_products(commit=False):
    def iterator():
        for data in PRODUCTS:
            yield Product(**data)

    if commit:
        Product.objects.bulk_create(iterator())

In [62]:
create_database_objects(commit=False, images_alone=True)

In [63]:
create_products(commit=False)

In [ ]:
USER_MODEL = get_user_model()

def create_users():
    users = [
        ['lucile@gmail.com', 'Lucile', 'Lopez', 'touparette'],
        ['pauline@gmail.com', 'Pauline', 'Matthieu', 'touparette']
    ]
    for user in users:
        try:
            user_object = USER_MODEL.objects.create_user(email=user[0], password=user[-1], first_name=user[1], last_name=user[2])
        except:
            print('User exists already')